In [1]:
import test_generator
import os
import sys
module_path = os.path.abspath(os.path.join('../../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from create_actionshist_keys import *
from actionshist_utils import *

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
input_path='/eos/user/l/llayer/AIErrorLogAnalysis/data/input/'
actionshist_path = '/eos/user/l/llayer/AIErrorLogAnalysis/data/actionshist/actionshistory_300719.json'
# Load the input
mode='tokens'
input_ml = pd.read_hdf(input_path + 'input.h5', 'frame')
sites = pd.read_hdf(input_path + 'input.h5', 'frame2')
codes = pd.read_hdf(input_path + 'input.h5', 'frame3')

# Load the actionshist
actionshist = load_data(actionshist_path)

In [4]:
input_ml.head()

,task_name,label,error,site,site_state,count,error_msg_tokenized,exit_codes,steps_counter,error_type,avg_w2v
0,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,0,"[-1, -1]","[T1_US_FNAL_Disk, T3_US_FNALLPC]","[bad, bad]","[1, 1]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
1,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,0,"[-1, -1]","[T3_US_FNALLPC, T1_US_FNAL_Disk]","[bad, bad]","[1, 1]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
2,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,0,"[-1, -1]","[T3_US_FNALLPC, T1_US_FNAL_Disk]","[bad, bad]","[1, 1]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
3,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,0,"[-1, -1, -1, -1, 85]","[T1_DE_KIT_Disk, T0_CH_CERN_MSS, T0_CH_CERN_Ex...","[bad, bad, bad, bad, good]","[1, 1, 1, 1, 1]","[nan, nan, nan, nan, [[14, 32, 67, 44, 7, 6, 8...","[nan, nan, nan, nan, [85, 8021, 99999, 85]]","[nan, nan, nan, nan, [2, 2, 2, 2]]","[nan, nan, nan, nan, [CMSSWStepFailure, Fatal ...","[nan, nan, nan, nan, [[-0.224627062678, 0.2987..."
4,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,0,"[50664, -1, -1]","[T2_DE_RWTH, T2_DE_RWTH, T2_CH_CERN]","[good, good, good]","[2, 1, 1]","[[[17, 1, 6, 189, 280, 179, 7, 341, 3, 140, 13...","[[50664, 143, 50115, 99999, 143], nan, nan]","[[0, 3, 3, 3, 3], nan, nan]","[[PerformanceKill, CMSSWStepFailure, BadFWJRXM...","[[[-0.298326134682, 1.518840909, 0.90024179220..."


In [31]:
import input_batch_generator as gen
import index 
sites_index, codes_index = index.to_index(list(sites['site']), list(codes['error']))
padding_size = 50
generator = gen.InputBatchGenerator(input_ml, 'label', codes_index, sites_index, padding_size, batch_size=100, 
                                   averaged = True, first_only = True)

In [32]:
batch = generator.msg_batch(0,100)

In [36]:
test = batch[0]

In [40]:
masked = np.ma.masked_equal(test, 0)

In [52]:
mean = masked.mean(axis=2)

In [53]:
mean[43][7]

masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True],
       fill_value=1e+20,
            dtype=float64)

In [11]:
test_generator.test(input_ml, actionshist, codes, sites, batch_test_w2v = True)

KeyboardInterrupt: 

In [32]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') 
        and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('input_ml', 104624840),
 ('actionshist', 41609160),
 ('sites', 12638),
 ('sites_index', 12568),
 ('codes', 7672),
 ('codes_index', 3352),
 ('action_encoded', 120),
 ('expand_key', 120),
 ('get_exit_codes', 120),
 ('get_keys', 120),
 ('get_sites', 120),
 ('load_data', 120),
 ('loop_over_sites', 120),
 ('map_to_key', 120),
 ('memory_encoded', 120),
 ('memory_fnc', 120),
 ('merge_labels', 120),
 ('set_binary_labels', 120),
 ('set_labels', 120),
 ('splitting_encoded', 120),
 ('splitting_fnc', 120),
 ('xrootd_encoded', 120),
 ('xrootd_fnc', 120),
 ('actionshist_path', 118),
 ('input_path', 86),
 ('module_path', 80),
 ('swan_spark_conf', 64),
 ('gen', 56),
 ('np', 56),
 ('pd', 56),
 ('mode', 43),
 ('x', 38)]